In [1]:
import math
import sklearn
import numpy

import theano
import theano.tensor as T
import lasagne

import json

import pandas as pd
from sklearn.cross_validation import train_test_split

/home/flerchy/.virtualenvs/ml/local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
good = open("../scripts/codestyle_stats/fine_res.json", "r")
bad = open("../scripts/codestyle_stats/shit_res.json", "r")

In [3]:
bad_data = bad.read().replace('\n', ' ')
bad_dict = json.loads(bad_data)

In [4]:
good_data = good.read().replace('\n', ' ')
good_dict = json.loads(good_data)

In [5]:
print len(bad_dict)

608


In [6]:
print len(good_dict)
print [0]*75

2540
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [7]:
X_train = []
for i in range(0, len(bad_dict)):
    s = []
    if ('errors' in bad_dict[i]):
        #print bad_dict[i]['errors']
        s.extend(numpy.array(bad_dict[i]['errors'].values()))
        #print s
    else:
        s.extend([0]*(len(bad_dict[0]['errors'])))
    #print s
    #print s
    for key, value in bad_dict[i].items():
         if key not in ('errors', 'filename'):
            s.append(numpy.array(int(value)))
    #print data
    #print s
    X_train.append(numpy.array(s, dtype=numpy.int32))
# for i in range(0, len(X_train)):
#     for elem in X_train[i]:
#         if not (isinstance( elem, int )):
#             print i
#             print type(elem)
#print X_train

In [8]:
for i in range(0, len(bad_dict)):
    s = []
    if ('errors' in good_dict[i]):
        #print bad_dict[i]['errors']
        s.extend(numpy.array(good_dict[i]['errors'].values()))
        #print s
    else:
        s.extend([0]*(len(bad_dict[0]['errors'])))
    #print s
    #print s
    for key, value in good_dict[i].items():
         if key not in ('errors', 'filename'):
            s.append(numpy.array(int(value)))
            #print data
    #print s
    X_train.append(numpy.array(s, dtype=numpy.int32))

In [9]:
X_train = numpy.asarray(X_train)
print X_train

[ array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0], dtype=int32)
 array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  2,  0, 55,  0,  2], dtype=int32)
 array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  1,  0, 19

In [10]:
y_train = numpy.array([1]*len(bad_dict) + [0]* len(bad_dict), dtype=numpy.int64)

In [11]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.4)
for s in (X_train, X_val, y_train, y_val):
    print(s.shape)

(729,)
(487,)
(729,)
(487,)


In [12]:
print numpy.shape(y_train)

(729,)


In [13]:
print numpy.shape(X_train)

(729,)


In [44]:
def build_network(input_var=None):
    l_in = lasagne.layers.InputLayer(shape=(75),
                                     input_var=input_var)
    
    l_in_drop = lasagne.layers.DropoutLayer(l_in, p = 0.5)
    
    l_hid = lasagne.layers.DenseLayer(l_in_drop, num_units=80, 
                                       nonlinearity=lasagne.nonlinearities.rectify,
                                       W=lasagne.init.GlorotUniform())
    l_out = lasagne.layers.DenseLayer(l_hid, num_units=2, 
                                      nonlinearity=lasagne.nonlinearities.softmax)
    
    return l_out

In [45]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    #print inputs
    #print targets
    if shuffle:
        indices = numpy.arange(len(inputs))
        numpy.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

In [48]:
input_var = T.ivector('inputs')
target_var = T.ivector('targets')

In [49]:
network = build_network(input_var)

TypeError: 'int' object is not iterable

In [40]:
prediction = lasagne.layers.get_output(network)
loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
loss = loss.mean()

In [41]:
params = lasagne.layers.get_all_params(network, trainable=True)
updates = lasagne.updates.nesterov_momentum(loss, params,
                                            learning_rate=0.01,
                                            momentum=0.9)
test_prediction = lasagne.layers.get_output(network, deterministic=True)
test_loss = lasagne.objectives.categorical_crossentropy(
                                                    test_prediction,
                                                    target_var)
test_loss = test_loss.mean()

In [42]:
train_fn = theano.function([input_var, target_var], loss)
val_fn = theano.function([input_var, target_var], test_loss)

In [43]:
for epoch in range(500):
    train_err = 0
    train_batches = 0
    print "123"
    for batch in iterate_minibatches(X_train, y_train, 100, shuffle=True):   
        inputs, targets = batch
        print type(inputs)
        print type(targets)
        train_err += train_fn(inputs, targets)
        train_batches += 1
    val_err = 0
    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(X_val, y_val, 100, shuffle=False):
        inputs, targets = batch
        err, acc = val_fn(inputs, targets)
        val_err += err
        val_acc += acc
        val_batches += 1
    print train_err / train_batches
    print val_err / val_batches

123
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>


AttributeError: Bad input argument to theano function with name "<ipython-input-42-477f93ffe1b2>:1" at index 0 (0-based).  
Backtrace when that variable is created:

  File "/home/flerchy/.virtualenvs/ml/local/lib/python2.7/site-packages/ipykernel/kernelbase.py", line 235, in dispatch_shell
    handler(stream, idents, msg)
  File "/home/flerchy/.virtualenvs/ml/local/lib/python2.7/site-packages/ipykernel/kernelbase.py", line 399, in execute_request
    user_expressions, allow_stdin)
  File "/home/flerchy/.virtualenvs/ml/local/lib/python2.7/site-packages/ipykernel/ipkernel.py", line 196, in do_execute
    res = shell.run_cell(code, store_history=store_history, silent=silent)
  File "/home/flerchy/.virtualenvs/ml/local/lib/python2.7/site-packages/ipykernel/zmqshell.py", line 533, in run_cell
    return super(ZMQInteractiveShell, self).run_cell(*args, **kwargs)
  File "/home/flerchy/.virtualenvs/ml/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2717, in run_cell
